In [1]:
from fireworks.core.launchpad import LaunchPad

In [2]:
lpad = LaunchPad.auto_load()
db = lpad.connection[lpad.name]

In [14]:
workflows = db['workflows']
fireworks = db['fireworks']
launches = db['launches']

In [12]:
list(workflows.find({"state": "FIZZLED"}))

[{'_id': ObjectId('63ef0a11178fdf74b66992c2'),
  'links': {'19944': []},
  'parent_links': {},
  'nodes': [19944],
  'metadata': {},
  'state': 'FIZZLED',
  'name': 'MiniRun1_1E19_RHC_rock_split4',
  'created_on': datetime.datetime(2023, 2, 17, 5, 1, 5, 254000),
  'updated_on': datetime.datetime(2023, 2, 19, 7, 58, 22, 908000),
  'fw_states': {'19944': 'FIZZLED'}}]

In [15]:
list(fireworks.find({"state": "FIZZLED"}))

[{'_id': ObjectId('63ef0a11178fdf74b66992c1'),
  'spec': {'runner': 'SimFor2x2_v1_GenieEdep',
   'base_env': 'MiniRun1_1E19_RHC_rock_split4',
   'env': {'ARCUBE_INDEX': '19943'},
   '_fworker': 'fworker_edep_sim',
   '_tasks': [{'_fw_name': 'RepoRunner'}]},
  'fw_id': 19944,
  'created_on': '2023-02-17T05:01:05.254820',
  'updated_on': '2023-02-19T07:58:22.908321',
  'launches': [24366],
  'archived_launches': [19939, 23814],
  'state': 'FIZZLED',
  'name': 'MiniRun1_1E19_RHC_rock_split4'}]

In [18]:
launches.find_one({"launch_id": 24366})

{'_id': ObjectId('63f1d53dd6bbc65dde8b6b7b'),
 'fworker': {'name': 'fworker_edep_sim',
  'category': '',
  'query': '{}',
  'env': {}},
 'fw_id': 19944,
 'launch_dir': '/global/cscratch1/sd/mkramer/fw_launches',
 'host': 'nid00761',
 'ip': '10.128.2.254',
 'trackers': [],
 'action': {'stored_data': {'_message': 'runtime error during task',
   '_task': {'_fw_name': 'RepoRunner'},
   '_exception': {'_stacktrace': 'Traceback (most recent call last):\n  File "/global/cfs/cdirs/dune/users/mkramer/mywork/fireworks4dune/fw.venv/lib/python3.9/site-packages/fireworks/core/rocket.py", line 261, in run\n    m_action = t.run_task(my_spec)\n  File "/global/cfs/cdirs/dune/users/mkramer/mywork/fireworks4dune/fw4dune_tasks/repo_runner.py", line 44, in run_task\n    check_call(runner[\'cmd\'],\n  File "/global/common/software/nersc/cori-2022q1/sw/python/3.9-anaconda-2021.11/lib/python3.9/subprocess.py", line 373, in check_call\n    raise CalledProcessError(retcode, cmd)\nsubprocess.CalledProcessError: 

HACK: In `/global/cscratch1/sd/mkramer/logs.edep_sim_job_fw`, `rg "globalIdx is 19943"`
(need to include SLURM job/task ID and/or log file path in Mongo DB! cf. FWAction)

That tells us that the log file is `/global/cscratch1/sd/mkramer/logs.edep_sim_job_fw/125115/output-125115.31.txt`

Underlying failure: `gevgen_fnal: KineUtils.cxx:325: genie::Range1D_t genie::utils::kinematics::InelQ2Lim_W(double, double, double, double, double): Assertion ``s>0' failed.`

Rerun it interactively:

```bash
# on perlmutter:
cd ~mkramer/dunescratch/mywork/fireworks4dune
source admin/load_fireworks.sh
lpad rerun_fws -s FIZZLED
rlaunch -w fw_config/fworker_edep_sim.yaml singleshot | tee straggler.txt
# later move straggler.txt to $CSCRATCH/logs.edep_sim_job_fw
```

Crashed again. Hacked around it by changing the seed from 19944 to 199440. Then it went through.

Perlmutter times (login node, 1 process):
```
gevgen_fnal 99% 1103236 8:30.63
gntpc 96% 476908 0:01.99
edep-sim 94% 889516 12:22.87
python3 108% 418752 0:36.18
```

Cori times (batch node, 64 processes):
```
gevgen_fnal 48% 1105400 1:04:45
gntpc 89% 474708 0:05.77
edep-sim 97% 804096 39:35.00
python3 33% 399152 6:16.99
```

Expect the Perlmutter advantage to be less dramatic when we're running 256 processes per node.

In [19]:
1e19 / (6.5e13 * 3000 * 24)

2.1367521367521367